In [4]:
from pytc2.sistemas_lineales import plot_plantilla
import sympy as sp
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
from scipy.signal import medfilt
import scipy.io as sio
from scipy.interpolate import CubicSpline

In [5]:

# Cargar la señal ECG
fs_ecg = 1000 # Hz

# para listar las variables que hay en el archivo
sio.whosmat('ECG_TP4.mat')
mat_struct = sio.loadmat('./ECG_TP4.mat')

ecg = mat_struct['ecg_lead'].flatten()
print('Tamaño de la señal ECG:', ecg.shape)
N = len(ecg)
print('Número de muestras:', N)


Tamaño de la señal ECG: (1129116,)
Número de muestras: 1129116


In [6]:
%matplotlib qt

# Parámetros de ventana
window_duration = 1  # segundos
window_size = int(fs * window_duration)  # muestras por ventana
step_size = window_size  # sin superposición, podés reducirlo para ventanas solapadas

# Crear listas para almacenar tiempos y medianas
median_times = []
median_values = []

# Recorrer la señal en bloques
for start in range(0, len(ecg) - window_size + 1, step_size):
    end = start + window_size
    window = ecg[start:end]
    median = np.median(window)
    time = (start + end) / 2 / fs  # tiempo central de la ventana
    median_times.append(time)
    median_values.append(median)

# Convertir a arrays
median_times = np.array(median_times)
median_values = np.array(median_values)

# Interpolación cúbica
cs = CubicSpline(median_times, median_values)

# Tiempo continuo para la spline (toda la duración de la señal)
t = np.arange(len(ecg)) / fs_ecg
trend = cs(t)

# Graficar
plt.figure(figsize=(12, 4))
plt.plot(t, ecg, label='Señal original', alpha=0.6)
plt.plot(t, trend, label='Tendencia (splines sobre medianas)', linewidth=2, color='orange')
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud')
plt.title('ECG con línea de tendencia suave')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Graficar
plt.figure(figsize=(12, 4))
plt.plot(t, ecg-trend, label='Señal original', alpha=0.6)
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud')
plt.title('ECG con línea de tendencia suave')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
%matplotlib qt
ecg_one_lead = ecg
cant_muestras = ecg_one_lead.shape[0]
fig_sz_x = 10
fig_sz_y = 5
fig_dpi = 100
demora = 0
ECG_f_win = ecg-trend
regs_interes = ( 
        np.array([5, 5.2]) *60*fs, # minutos a muestras
        np.array([12, 12.4]) *60*fs, # minutos a muestras
        np.array([15, 15.2]) *60*fs, # minutos a muestras
        )

for ii in regs_interes:
    
    # intervalo limitado de 0 a cant_muestras
    zoom_region = np.arange(np.max([0, ii[0]]), np.min([cant_muestras, ii[1]]), dtype='uint')
    
    plt.figure(figsize=(fig_sz_x, fig_sz_y), dpi= fig_dpi, facecolor='w', edgecolor='k')
    plt.plot(zoom_region, ecg_one_lead[zoom_region], label='ECG', linewidth=2)
    #plt.plot(zoom_region, ECG_f_butt[zoom_region], label='Butter')
    plt.plot(zoom_region, ECG_f_win[zoom_region + demora], label='Win')
    
    plt.title('ECG filtering example from ' + str(ii[0]) + ' to ' + str(ii[1]) )
    plt.ylabel('Adimensional')
    plt.xlabel('Muestras (#)')
    
    axes_hdl = plt.gca()
    axes_hdl.legend()
    axes_hdl.set_yticks(())
            
    plt.show()

    
regs_interes = ( 
        [4000, 5500], # muestras
        [10e3, 11e3], # muestras
        )

for ii in regs_interes:
    
    # intervalo limitado de 0 a cant_muestras
    zoom_region = np.arange(np.max([0, ii[0]]), np.min([cant_muestras, ii[1]]), dtype='uint')
    
    plt.figure(figsize=(fig_sz_x, fig_sz_y), dpi= fig_dpi, facecolor='w', edgecolor='k')
    plt.plot(zoom_region, ecg_one_lead[zoom_region], label='ECG', linewidth=2)
    #plt.plot(zoom_region, ECG_f_butt[zoom_region], label='Butter')
    plt.plot(zoom_region, ECG_f_win[zoom_region + demora], label='Win')
    
    plt.title('ECG filtering example from ' + str(ii[0]) + ' to ' + str(ii[1]) )
    plt.ylabel('Adimensional')
    plt.xlabel('Muestras (#)')
    
    axes_hdl = plt.gca()
    axes_hdl.legend()
    axes_hdl.set_yticks(())
            
    plt.show()